TODO:
- Make code that plots total energy hist for day for each appliance 
- Make code that plots for each app by house
- Fix data for homes that have different ...?
- Table of appliances in each home, date of first/last measurement.
- Plot time series of data for some home for some day, showing main and individual appliances.
- Show histogram of energy used in each home.
- Show histogram of energy used by appliances in each home.
- Replicate table 3.4 from Chaoyun's paper
- Full day of data for each home on each day? Day 1: 20 homes with complete data, Day 2:...
- Solar panels - ignore these homes (11 and 21)
- Mystery kettle
- House 12 has kettle mistake (says 6 when it should be 4). See 
- Check Chaoyun's code

In [68]:
import os
import re
import pandas as pd
import numpy as np
import time
import datetime

import sqlite3

from ggplot import *
import matplotlib.pyplot as plt
import matplotlib

from utils import *
import refit_io
from get_activations import *

In [69]:
%matplotlib inline
matplotlib.style.use('ggplot')

In [121]:
dir_proj = '/Users/sipola/Google Drive/education/coursework/graduate/edinburgh/dissertation/thesis'
dir_data = os.path.join(dir_proj, 'data')
dir_run = os.path.join(dir_proj, 'run', str(datetime.date.today()))

In [131]:
house_ids = range(1, 22)
house_ids.remove(14)  # no house 14

In [3]:
params_appliance = get_params_appliance()
app_df = pd.read_csv(os.path.join(dir_data, 'appliances.csv'))
refit_data_path = os.path.join(dir_data, 'REFITPowerData111215')

In [7]:
reload(refit_io)
dir_refit = os.path.join(dir_data, 'refit')

In [259]:
# conn = sqlite3.connect(os.path.join(dir_data, 'refit.db'))
# data_import.save_refit_data_to_sqlite(conn, refit_data_path, nrows=None)
io.save_refit_data(refit_data_path, 'numpy', dir_refit=dir_refit, dir_data=dir_data, nrows=None)
# conn.close()

In [260]:
len(np.load(os.path.join(dir_refit, 'house1', 'timestamp.npy')))

7633070

In [278]:
app_df

,House,ApplianceNum,ApplianceOrig,Appliance
0,1,1,Fridge,fridge
1,1,2,Chest Freezer,chest freezer
2,1,3,Upright Freezer,upright freezer
3,1,4,Tumble Dryer,tumble dryer
4,1,5,Washing Machine,washing machine
5,1,6,Dishwasher,dishwasher
6,1,7,Computer Site,computer site
7,1,8,Television Site,television site
8,1,9,Electric Heater,electric heater
9,2,1,Fridge-Freezer,fridge-freezer


In [105]:
def get_house_app_tuples(pattern, return_pd=False):
    is_match = app_df['ApplianceOrig'].str.lower().str.contains(pattern)
    if return_pd:
        return app_df.loc[is_match]
    house_to_app = app_df.loc[is_match][['House', 'ApplianceNum']].values
    return [(h, a) for h, a in house_to_app]

def load_app(house_id, app_num):
    return np.load(os.path.join(dir_refit,
                                'house{}'.format(house_id),
                                'appliance{}.npy'.format(app_num)))
    
def load_ts(house_id):
    return np.load(os.path.join(dir_refit,
                                'house{}'.format(house_id),
                                'timestamp.npy'))

def get_app_name(house_id, app_num):
    return app_df[(app_df['House'] == house_id) & (app_df['ApplianceNum'] == app_num)]['Appliance'].values[0]

get_house_app_tuples('fridge')
get_house_app_tuples('kettle')
get_house_app_tuples('washing machine')

# fridge_dat = []
# for (house_id, app_num) in get_house_app_tuples('fridge'):
#     np.sample()

[(1, 5),
 (2, 2),
 (3, 6),
 (4, 4),
 (4, 5),
 (5, 3),
 (6, 2),
 (7, 5),
 (8, 4),
 (9, 3),
 (10, 5),
 (11, 3),
 (13, 3),
 (15, 3),
 (16, 5),
 (17, 4),
 (18, 5),
 (19, 2),
 (20, 4),
 (21, 3)]

In [ ]:
def get_day_idx(dt, ts_series):
    ts0 = dt2ts(dt)
    ts1 = dt2ts(dt + datetime.timedelta(days=1))
    ts_series = pd.DataFrame(ts_series).set_index(0).index
    return (ts_series >= ts0) & (ts_series < ts1)

# get_day_timestamps(datetime.datetime(2015,1,1), load_ts(1))

def plot_day(house_id, dt, savefile=None):
    ts_series = load_ts(house_id)
    idx = get_day_idx(dt, ts_series)
    df = pd.DataFrame({'timestamp': ts_series[idx]})
    for app_num in range(10):
        app_name = get_app_name(house_id, app_num)
        df[app_name] = load_app(house_id, app_num)[idx]

    df['timestamp'] = pd.to_datetime(df.timestamp, unit='s', utc=True)
    df.set_index('timestamp', inplace=True)
    # df = df.tz_localize('GMT').tz_convert('Europe/London')
    df.plot(figsize=(11,8))
    if savefile is not None:
        plt.savefig(savefile)

dt_start = datetime.datetime(2013,11,1)
for house_id in house_ids:
    house_dir = os.path.join(dir_run, 'daily_power', 'house{}'.format(house_id))
    if not os.path.exists(house_dir):
        os.makedirs(house_dir)
    for days_to_add in np.array(range(20))*30:
        dt = dt_start + datetime.timedelta(days=days_to_add)
        savefile = os.path.join(house_dir, '{}.pdf'.format(str(dt.date())))
        try:
            plot_day(house_id, dt, savefile)
        except TypeError:
            pass
            # print 'No data to plot for house{}_{}.pdf'.format(house_id, str(dt.date()))
        plt.close()

No data to plot for house1_2014-01-30.pdf
No data to plot for house1_2014-03-01.pdf
No data to plot for house1_2015-05-25.pdf
No data to plot for house2_2013-11-01.pdf
No data to plot for house2_2013-12-01.pdf
No data to plot for house2_2013-12-31.pdf
No data to plot for house2_2014-01-30.pdf
No data to plot for house3_2014-01-30.pdf
No data to plot for house3_2014-03-01.pdf
No data to plot for house4_2014-03-31.pdf
No data to plot for house6_2013-11-01.pdf
No data to plot for house6_2014-01-30.pdf
No data to plot for house6_2014-03-01.pdf
No data to plot for house7_2014-01-30.pdf
No data to plot for house7_2014-03-01.pdf
No data to plot for house7_2015-05-25.pdf
No data to plot for house8_2014-01-30.pdf
No data to plot for house8_2014-03-01.pdf
No data to plot for house8_2015-05-25.pdf
No data to plot for house9_2013-11-01.pdf
No data to plot for house9_2013-12-01.pdf
No data to plot for house9_2014-01-30.pdf
No data to plot for house9_2014-03-01.pdf
No data to plot for house10_2013-1

In [138]:
np.array(range(41))*15
# datetime.datetime(2013,11,1) + datetime.timedelta(days=600)

array([  0,  15,  30,  45,  60,  75,  90, 105, 120, 135, 150, 165, 180,
       195, 210, 225, 240, 255, 270, 285, 300, 315, 330, 345, 360, 375,
       390, 405, 420, 435, 450, 465, 480, 495, 510, 525, 540, 555, 570,
       585, 600])

In [201]:
def query(string):
    conn = sqlite3.connect(os.path.join(dir_data, 'refit.db'))
    df = pd.read_sql(string, conn)
    conn.close()
    return df


def tt():
    return time.time()


def runtime(t0):
    print '{1:.2g}'.format((tt() - t0) / 60)

In [171]:
# Test
query('SELECT * FROM power LIMIT 3')

,Timestamp,House,Appliance0,Appliance1,Appliance2,Appliance3,Appliance4,Appliance5,Appliance6,Appliance7,Appliance8,Appliance9
0,1381323977,1,523,74,0,69,0,0,0,0,0,1
1,1381323990,1,526,75,0,69,0,0,0,0,0,1
2,1381324005,1,540,74,0,68,0,0,0,0,0,1


In [172]:
# Initialize dict of dataframes for exploratory data analysis
dfs = {}

In [173]:
# Get min and max of timestamp for each home
t0 = tt()
dfs['ts_min_max'] = query('SELECT House, MIN(Timestamp) AS TimestampMin, MAX(Timestamp) AS TimestampMax FROM power GROUP BY House')
runtime(t0)

In [189]:
pd.DataFrame.from_dict({'House': dfs['ts_min_max']['House'],
                        'DatetimeMin': [str(i.date()) for i in ts2dt(dfs['ts_min_max']['TimestampMin'])],
                        'DatetimeMax': [str(i.date()) for i in ts2dt(dfs['ts_min_max']['TimestampMax'])]})

,DatetimeMax,DatetimeMin,House
0,2015-07-10,2013-10-09,1
1,2015-05-28,2013-09-17,2
2,2015-06-02,2013-09-25,3
3,2015-07-07,2013-10-11,4
4,2015-07-06,2013-09-26,5
5,2015-06-28,2013-11-28,6
6,2015-07-08,2013-11-01,7
7,2015-05-10,2013-11-01,8
8,2015-07-08,2013-12-17,9
9,2015-06-30,2013-11-20,10


In [210]:
# Get appliance statistics for each home. To be used in get_activations().
with sqlite3.connect(os.path.join(dir_data, 'refit.db')) as conn:
    conn.create_aggregate('stdev', 1, StdevFunc)

# Build query string starting with column definitions.
query_str = ['House']
for app_df in range(10):
    query_str.append('MAX(Appliance{0}) AS MaxOnPower, AVG(Appliance{0}) AS MeanPower, STDEV(Appliance{0}) AS StddevPower'.format(app))
query_str = ','.join(query_str)
query_str = 'SELECT ' + query_str + ' FROM power GROUP BY House'
conn = sqlite3.connect(os.path.join(dir_data, 'refit.db'))

# Run query.
t0 = tt()
conn = sqlite3.connect(os.path.join(dir_data, 'refit.db'))
conn.create_aggregate('stdev', 1, StdevFunc)
df = pd.read_sql(query_str, conn)
dfs['app_stats'] = query(query_str)
conn.close()
runtime(t0)

DatabaseError: Execution failed on sql 'SELECT House,MAX(Appliance0) AS MaxOnPower, AVG(Appliance0) AS MeanPower, STDEV(Appliance0) AS StddevPower,MAX(Appliance1) AS MaxOnPower, AVG(Appliance1) AS MeanPower, STDEV(Appliance1) AS StddevPower,MAX(Appliance2) AS MaxOnPower, AVG(Appliance2) AS MeanPower, STDEV(Appliance2) AS StddevPower,MAX(Appliance3) AS MaxOnPower, AVG(Appliance3) AS MeanPower, STDEV(Appliance3) AS StddevPower,MAX(Appliance4) AS MaxOnPower, AVG(Appliance4) AS MeanPower, STDEV(Appliance4) AS StddevPower,MAX(Appliance5) AS MaxOnPower, AVG(Appliance5) AS MeanPower, STDEV(Appliance5) AS StddevPower,MAX(Appliance6) AS MaxOnPower, AVG(Appliance6) AS MeanPower, STDEV(Appliance6) AS StddevPower,MAX(Appliance7) AS MaxOnPower, AVG(Appliance7) AS MeanPower, STDEV(Appliance7) AS StddevPower,MAX(Appliance8) AS MaxOnPower, AVG(Appliance8) AS MeanPower, STDEV(Appliance8) AS StddevPower,MAX(Appliance9) AS MaxOnPower, AVG(Appliance9) AS MeanPower, STDEV(Appliance9) AS StddevPower FROM po

In [ ]:
dfs['app_stats']